In [ ]:
-- given corr_ids, get data (secondary)

%default startDate1 '2020/08/20-21:00:00'
%default endDate1 '2020/08/20-21:15:00'

%default corrID1 '7baf4b0653052'
-- %default corrID2 'bee179d7bbdf5'

%default server1 'acserv_acriskserv'
%default server2 ' compliancepaymentserv'


table1 = LOAD 'tmp' using RootTransactionLoader('all', 'paypal','$server1', 'null','$startDate1', '$endDate1', '100','null','null','corr_id_','$corrID1') as (env:bytearray, pool:bytearray, machine:bytearray, threadId:int, calClass:bytearray, time:long, type:bytearray,  name:bytearray, status:bytearray, duration:float, dataMap:map[], records:bag{(record:bytearray)}, key:bytearray, colo:bytearray);


-- table1 = LOAD 'tmp' using RootTransactionLoader('all', 'paypal','$server1', 'null','$startDate1', '$endDate1', '100') as (env:bytearray, pool:bytearray, machine:bytearray, threadId:int, calClass:bytearray, time:long, type:bytearray,  name:bytearray, status:bytearray, duration:float, dataMap:map[], records:bag{(record:bytearray)}, key:bytearray, colo:bytearray);

-- table2 = LOAD 'tmp' using RootTransactionLoader('all', 'paypal','$server2', 'null','$startDate2', '$endDate2', '100') as (env:bytearray, pool:bytearray, machine:bytearray, threadId:int, calClass:bytearray, time:long, type:bytearray,  name:bytearray, status:bytearray, duration:float, dataMap:map[], records:bag{(record:bytearray)}, key:bytearray, colo:bytearray);

-- records_all = Group table1 All;
-- rec_count = FOREACH records_all Generate COUNT(table1);

-- comb = UNION table1,table2;
-- test2 = foreach table1 generate CONCAT(name, '_',status);

test1 = FOREACH table1 {
           PXP  = FILTER records BY (record MATCHES '.*\tEXEC\t.*'OR record MATCHES '.*\tBIZ\t.*'OR record MATCHES '.*\tSTART_DML\t.*');
            PXP = FOREACH PXP GENERATE GET_FIELDS($0) as rec:tuple(mClass:bytearray, rType:bytearray, rName:bytearray, rStatus:bytearray, rDuration:float, rData:bytearray, rDataMap:map[]);
                      
          PXP = FOREACH PXP GENERATE rec.mClass AS (mClass:bytearray),rec.rType AS (rType:bytearray),rec.rName AS (rName:bytearray),rec.rStatus AS (rStatus:bytearray),rec.rData AS (rData:bytearray),rec.rDataMap AS (rDataMap:map[]);
--           index1 = foreach PXP generate CONCAT(rType,rName) as key, mClass,rType,rName,rStatus,rData,rDataMap;

--           GENERATE PXP,rName as key;
          GENERATE PXP;

         };
   
-- group_data1 = GROUP test1 by corrID;

-- test2 =FOREACH table2 {
-- --            PXP  = FILTER records BY (record MATCHES '.*\t2\t.*'OR record MATCHES '.*\t3\t.*'OR record MATCHES '.*\t1\t.*'OR record MATCHES '.*\t4\t.*');

--            PXP2  = FILTER records BY (record MATCHES '.*\tEXEC\t.*'OR record MATCHES '.*\tBIZ\t.*'OR record MATCHES '.*\tSTART_DML\t.*');
--            PXP2 = FOREACH PXP2 GENERATE GET_FIELDS($0) as rec:tuple(mClass:bytearray, rType:bytearray, rName:bytearray, rStatus:bytearray, rDuration:float, rData:bytearray, rDataMap:map[]);
           
--            PXP2 = FOREACH PXP2 GENERATE rec.mClass AS (mClass:bytearray),rec.rType AS (rType:bytearray),rec.rName AS (rName:bytearray),rec.rStatus AS (rStatus:bytearray),rec.rData AS (rData:bytearray),rec.rDataMap AS (rDataMap:map[]);
-- --           index2 = foreach PXP2 generate CONCAT(rType,rName) as key, mClass,rType,rName,rStatus,rData,rDataMap;
-- --           GENERATE PXP2,dataMap#'corr_id_' as (corrID:bytearray);
--           GENERATE PXP2,rName as key;

--           GENERATE index2;


--          };
-- group_data2 = GROUP test2 by corrID;

-- cogroup_data = COGROUP test1 by key, test2 by key;

-- diff_data = FOREACH cogroup_data GENERATE DIFF(test1,test2);
-- result = COGROUP test1 by $0, test2 by $0;


T = LIMIT test1 20;
store T into '$output' using PigStorage(',');  

In [ ]:
table0 = LOAD 'tmp' using RootTransactionLoader('null', 'paypal','acserv_acriskserv', 'null', '2020/08/18-15:00:00', '2020/08/18-15:15:00', '100','null','null','corr_id_','10023d1051f9') as (env:bytearray, pool:bytearray, machine:bytearray, threadId:int, calClass:bytearray, time:long, type:bytearray,  name:bytearray, status:bytearray, duration:float, dataMap:map[], records:bag{(record:bytearray)}, key:bytearray, colo:bytearray);
        
paps =FOREACH table0 {
           PXP  = FILTER records BY (record MATCHES '.*	2	.*'OR record MATCHES '.*	3	.*'OR record MATCHES '.*	1	.*'OR record MATCHES '.*	4	.*');
           PXP = FOREACH PXP GENERATE GET_FIELDS($0) as rec:tuple(mClass:bytearray, rType:bytearray, rName:bytearray, rStatus:bytearray, rDuration:float, rData:bytearray, rDataMap:map[]);
           PXP = FOREACH PXP GENERATE  rec.mClass AS (mClass:bytearray),rec.rType AS (rType:bytearray),rec.rName AS (rName:bytearray),rec.rStatus--rec.mClass,rec.rType--,rec.rName,rec.rStatus,rec.rStatus,rec.rDuration,rec.rData,rec.rDataMap,rec.rTime;  --as (rTime:long);
           AS (rStatus:bytearray);
       GENERATE dataMap#'corr_id_' as (corrID:bytearray),PXP;
         }
         
group0 = GROUP paps by corrID;

 
table1 = LOAD 'tmp' using RootTransactionLoader('null', 'paypal','acserv_acriskserv', 'null', '2020/08/18-15:00:00', '2020/08/18-15:15:00', '100','null','null','corr_id_','1003cffc0d62') as (env:bytearray, pool:bytearray, machine:bytearray, threadId:int, calClass:bytearray, time:long, type:bytearray,  name:bytearray, status:bytearray, duration:float, dataMap:map[], records:bag{(record:bytearray)}, key:bytearray, colo:bytearray);
        
paps =FOREACH table1 {
           PXP  = FILTER records BY (record MATCHES '.*	2	.*'OR record MATCHES '.*	3	.*'OR record MATCHES '.*	1	.*'OR record MATCHES '.*	4	.*');
           PXP = FOREACH PXP GENERATE GET_FIELDS($0) as rec:tuple(mClass:bytearray, rType:bytearray, rName:bytearray, rStatus:bytearray, rDuration:float, rData:bytearray, rDataMap:map[]);
           PXP = FOREACH PXP GENERATE  rec.mClass AS (mClass:bytearray),rec.rType AS (rType:bytearray),rec.rName AS (rName:bytearray),rec.rStatus--rec.mClass,rec.rType--,rec.rName,rec.rStatus,rec.rStatus,rec.rDuration,rec.rData,rec.rDataMap,rec.rTime;  --as (rTime:long);
           AS (rStatus:bytearray);
       GENERATE dataMap#'corr_id_' as (corrID:bytearray),PXP;
         }
         
group1 = GROUP paps by corrID;

 
table2 = LOAD 'tmp' using RootTransactionLoader('null', 'paypal','acserv_acriskserv', 'null', '2020/08/18-15:00:00', '2020/08/18-15:15:00', '100','null','null','corr_id_','1004a9750f17') as (env:bytearray, pool:bytearray, machine:bytearray, threadId:int, calClass:bytearray, time:long, type:bytearray,  name:bytearray, status:bytearray, duration:float, dataMap:map[], records:bag{(record:bytearray)}, key:bytearray, colo:bytearray);
        
paps =FOREACH table2 {
           PXP  = FILTER records BY (record MATCHES '.*	2	.*'OR record MATCHES '.*	3	.*'OR record MATCHES '.*	1	.*'OR record MATCHES '.*	4	.*');
           PXP = FOREACH PXP GENERATE GET_FIELDS($0) as rec:tuple(mClass:bytearray, rType:bytearray, rName:bytearray, rStatus:bytearray, rDuration:float, rData:bytearray, rDataMap:map[]);
           PXP = FOREACH PXP GENERATE  rec.mClass AS (mClass:bytearray),rec.rType AS (rType:bytearray),rec.rName AS (rName:bytearray),rec.rStatus--rec.mClass,rec.rType--,rec.rName,rec.rStatus,rec.rStatus,rec.rDuration,rec.rData,rec.rDataMap,rec.rTime;  --as (rTime:long);
           AS (rStatus:bytearray);
       GENERATE dataMap#'corr_id_' as (corrID:bytearray),PXP;
         }
         
group2 = GROUP paps by corrID;

 
table3 = LOAD 'tmp' using RootTransactionLoader('null', 'paypal','acserv_acriskserv', 'null', '2020/08/18-15:00:00', '2020/08/18-15:15:00', '100','null','null','corr_id_','100d84c7397e') as (env:bytearray, pool:bytearray, machine:bytearray, threadId:int, calClass:bytearray, time:long, type:bytearray,  name:bytearray, status:bytearray, duration:float, dataMap:map[], records:bag{(record:bytearray)}, key:bytearray, colo:bytearray);
        
paps =FOREACH table3 {
           PXP  = FILTER records BY (record MATCHES '.*	2	.*'OR record MATCHES '.*	3	.*'OR record MATCHES '.*	1	.*'OR record MATCHES '.*	4	.*');
           PXP = FOREACH PXP GENERATE GET_FIELDS($0) as rec:tuple(mClass:bytearray, rType:bytearray, rName:bytearray, rStatus:bytearray, rDuration:float, rData:bytearray, rDataMap:map[]);
           PXP = FOREACH PXP GENERATE  rec.mClass AS (mClass:bytearray),rec.rType AS (rType:bytearray),rec.rName AS (rName:bytearray),rec.rStatus--rec.mClass,rec.rType--,rec.rName,rec.rStatus,rec.rStatus,rec.rDuration,rec.rData,rec.rDataMap,rec.rTime;  --as (rTime:long);
           AS (rStatus:bytearray);
       GENERATE dataMap#'corr_id_' as (corrID:bytearray),PXP;
         }
         
group3 = GROUP paps by corrID;

 
table4 = LOAD 'tmp' using RootTransactionLoader('null', 'paypal','acserv_acriskserv', 'null', '2020/08/18-15:00:00', '2020/08/18-15:15:00', '100','null','null','corr_id_','10102689e97b') as (env:bytearray, pool:bytearray, machine:bytearray, threadId:int, calClass:bytearray, time:long, type:bytearray,  name:bytearray, status:bytearray, duration:float, dataMap:map[], records:bag{(record:bytearray)}, key:bytearray, colo:bytearray);
        
paps =FOREACH table4 {
           PXP  = FILTER records BY (record MATCHES '.*	2	.*'OR record MATCHES '.*	3	.*'OR record MATCHES '.*	1	.*'OR record MATCHES '.*	4	.*');
           PXP = FOREACH PXP GENERATE GET_FIELDS($0) as rec:tuple(mClass:bytearray, rType:bytearray, rName:bytearray, rStatus:bytearray, rDuration:float, rData:bytearray, rDataMap:map[]);
           PXP = FOREACH PXP GENERATE  rec.mClass AS (mClass:bytearray),rec.rType AS (rType:bytearray),rec.rName AS (rName:bytearray),rec.rStatus--rec.mClass,rec.rType--,rec.rName,rec.rStatus,rec.rStatus,rec.rDuration,rec.rData,rec.rDataMap,rec.rTime;  --as (rTime:long);
           AS (rStatus:bytearray);
       GENERATE dataMap#'corr_id_' as (corrID:bytearray),PXP;
         }
         
group4 = GROUP paps by corrID;

All_row = UNION group0, group1, group2, group3, group4; 
Result = ORDER All_row BY corrID ASC;  
store Result into '$output' using PigStorage(',');

-- table0 = LOAD 'tmp' using RootTransactionLoader('null', 'paypal','acserv_acriskserv', 'null', '2020/08/18-15:00:00', '2020/08/18-15:15:00', '100','null','null','corr_id_','10023d1051f9') as (env:bytearray, pool:bytearray, machine:bytearray, threadId:int, calClass:bytearray, time:long, type:bytearray,  name:bytearray, status:bytearray, duration:float, dataMap:map[], records:bag{(record:bytearray)}, key:bytearray, colo:bytearray);

-- table1 = LOAD 'tmp' using RootTransactionLoader('null', 'paypal','acserv_acriskserv', 'null', '2020/08/18-15:00:00', '2020/08/18-15:15:00', '100','null','null','corr_id_','1003cffc0d62') as (env:bytearray, pool:bytearray, machine:bytearray, threadId:int, calClass:bytearray, time:long, type:bytearray,  name:bytearray, status:bytearray, duration:float, dataMap:map[], records:bag{(record:bytearray)}, key:bytearray, colo:bytearray);
    
-- table2 = LOAD 'tmp' using RootTransactionLoader('null', 'paypal','acserv_acriskserv', 'null', '2020/08/18-15:00:00', '2020/08/18-15:15:00', '100','null','null','corr_id_','1004a9750f17') as (env:bytearray, pool:bytearray, machine:bytearray, threadId:int, calClass:bytearray, time:long, type:bytearray,  name:bytearray, status:bytearray, duration:float, dataMap:map[], records:bag{(record:bytearray)}, key:bytearray, colo:bytearray);

-- All_row = UNION table0, table1, table2; 

-- paps =FOREACH All_row {
--            PXP  = FILTER records BY (NOT(record MATCHES '-1') OR record MATCHES '.*	2	.*'OR record MATCHES '.*	3	.*'OR record MATCHES '.*	1	.*'OR record MATCHES '.*	4	.*');
--            PXP = FOREACH PXP GENERATE GET_FIELDS($0) as rec:tuple(mClass:bytearray, rType:bytearray, rName:bytearray, rStatus:bytearray, rDuration:float, rData:bytearray, rDataMap:map[]);
--            PXP = FOREACH PXP GENERATE  rec.mClass AS (mClass:bytearray),rec.rType AS (rType:bytearray),rec.rName AS (rName:bytearray),rec.rStatus AS (rStatus:bytearray);
--        GENERATE dataMap#'corr_id_' as (corrID:bytearray),flatten(PXP);
--          }
                                                                  
-- order_group = ORDER paps BY corrID ASC;  
-- Result = GROUP order_group by corrID;

-- store All_row into '$output' using PigStorage(',');
 

In [ ]:
-- get_CAL_records, use sample given by Anant
%default startDate '2020/06/22-00:00:00'
%default endDate '2020/06/22-00:15:00'
%default corrID 'b9b60c9b673fc'
    

paps = LOAD 'tmp' using RootTransactionLoader('all', 'paypal','acserv_acriskserv', 'null','$startDate', '$endDate', '100','null','null','corr_id_','$corrID') as (env:bytearray, pool:bytearray, machine:bytearray, threadId:int, calClass:bytearray, time:long, type:bytearray,  name:bytearray, status:bytearray, duration:float, dataMap:map[], records:bag{(record:bytearray)}, key:bytearray, colo:bytearray);

-- test = FOREACH paps GENERATE field1, field2, field2 as field2_copy;

test =FOREACH paps {
           PXP  = FILTER records BY (record MATCHES '.*\t2\t.*'OR record MATCHES '.*\t3\t.*'OR record MATCHES '.*\t1\t.*'OR record MATCHES '.*\t4\t.*');
           PXP = FOREACH PXP GENERATE GET_FIELDS($0) as rec:tuple(mClass:bytearray, rType:bytearray, rName:bytearray, rStatus:bytearray, rDuration:float, rData:bytearray, rDataMap:map[]);
           
           PXP = FOREACH PXP GENERATE rec.mClass AS (mClass:bytearray),rec.rType AS (rType:bytearray),rec.rName AS (rName:bytearray),rec.rStatus AS (rStatus:bytearray),rec.rData AS (rData:bytearray),rec.rDataMap AS (rDataMap:map[]);
--rec.mClass,rec.rType--,rec.rName,rec.rStatus,rec.rStatus,rec.rDuration,rec.rData,rec.rDataMap,rec.rTime;  --as (rTime:long);
           
        GENERATE FLATTEN(PXP),dataMap#'corr_id_';
        
         };
                                                                  
T = LIMIT test 100;
store T into '$output' using PigStorage(',');                                              
                                           

In [ ]:
-- Not working get_sql...

%default startDate1 '2020/06/30-00:00:00'
%default endDate1 '2020/06/30-01:00:00'
%default startDate2 '2020/06/30-00:00:00'
%default endDate2 '2020/06/30-01:00:00'

%default corrID 'ac65f81dd963e'

%default server1 'acserv_acriskserv'
%default server2 'activitylogserv'

%default machine 'slcfeel3542b'


table1 = LOAD 'tmp' using RootTransactionLoader('all', 'paypal','$server1', '$machine','$startDate1', '$endDate1', '100','null','null','corr_id_','$corrID') as (env:bytearray, pool:bytearray, machine:bytearray, threadId:int, calClass:bytearray, time:long, type:bytearray,  name:bytearray, status:bytearray, duration:float, dataMap:map[], records:bag{(record:bytearray)}, key:bytearray, colo:bytearray);


test =FOREACH table1 {
        PXP  = FILTER records BY (record MATCHES '.*\tEXEC\t.*'OR record MATCHES '.*\tBIZ\t.*'OR record MATCHES '.*\tSTART_DML\t.*');
        PXP2 = FOREACH PXP GENERATE GET_FIELDS($0) as rec:tuple(mClass:bytearray, rType:bytearray, rName:bytearray, rStatus:bytearray, rDuration:float, rData:bytearray, rDataMap:map[]);
        H = FOREACH PXP2 GENERATE GET_SQL(rec.rName) as sql;
        INS = FOREACH H GENERATE sql ;           
           
        GENERATE INS;
         };


                                                      
T = LIMIT test 100;
store T into '$output' using PigStorage(',');    

In [ ]:
-- get_CAL_records, use sample given by Anant, trying for 2 records's comparison!
%default startDate1 '2020/07/08-15:00:00'
%default endDate1 '2020/07/08-16:00:00'
%default startDate2 '2020/07/08-15:00:00'
%default endDate2 '2020/07/08-16:00:00'

%default corrID1 'slcfeel3542b'
-- %default corrID2 'bee179d7bbdf5'

%default server1 'acserv_acriskserv'
%default server2 'activitydataserv'

-- %default machine 'slcfeel3542b'
%default machine 'slcfeel6412b'

-- GET_SQL	Gets SQL statement given a SQL hash	GET_SQL(hash) as sql:bytearray
-- GET_SQL_NAME	Gets SQL's name given a SQL hash	GET_SQL_NAME(hash) as name:chararray


-- table1 = LOAD 'tmp' using RootTransactionLoader('all', 'paypal','$server1', '$machine','$startDate1', '$endDate1', '100','null','null','corr_id_','$corrID1') as (env:bytearray, pool:bytearray, machine:bytearray, threadId:int, calClass:bytearray, time:long, type:bytearray,  name:bytearray, status:bytearray, duration:float, dataMap:map[], records:bag{(record:bytearray)}, key:bytearray, colo:bytearray);

table1 = LOAD 'tmp' using RootTransactionLoader('all', 'paypal','$server1', 'null','$startDate1', '$endDate1', '100') as (env:bytearray, pool:bytearray, machine:bytearray, threadId:int, calClass:bytearray, time:long, type:bytearray,  name:bytearray, status:bytearray, duration:float, dataMap:map[], records:bag{(record:bytearray)}, key:bytearray, colo:bytearray);

table2 = LOAD 'tmp' using RootTransactionLoader('all', 'paypal','$server2', '$machine','$startDate2', '$endDate2', '100','null','null','corr_id_','$corrID1') as (env:bytearray, pool:bytearray, machine:bytearray, threadId:int, calClass:bytearray, time:long, type:bytearray,  name:bytearray, status:bytearray, duration:float, dataMap:map[], records:bag{(record:bytearray)}, key:bytearray, colo:bytearray);

-- records_all = Group table1 All;
-- rec_count = FOREACH records_all Generate COUNT(table1);

-- comb = UNION table1,table2;
-- test2 = foreach table1 generate CONCAT(name, '_',status);

test1 = FOREACH table1 {
           PXP  = FILTER records BY (record MATCHES '.*\tEXEC\t.*'OR record MATCHES '.*\tBIZ\t.*'OR record MATCHES '.*\tSTART_DML\t.*');
            PXP = FOREACH PXP GENERATE GET_FIELDS($0) as rec:tuple(mClass:bytearray, rType:bytearray, rName:bytearray, rStatus:bytearray, rDuration:float, rData:bytearray, rDataMap:map[]);
                      
          PXP = FOREACH PXP GENERATE rec.mClass AS (mClass:bytearray),rec.rType AS (rType:bytearray),rec.rName AS (rName:bytearray),rec.rStatus AS (rStatus:bytearray),rec.rData AS (rData:bytearray),rec.rDataMap AS (rDataMap:map[]);
          GENERATE dataMap#'corr_id_' as (corrID:bytearray),PXP;

         };
   
group_data = GROUP test1 by corrID;

-- test2 =FOREACH table2 {
-- --            PXP  = FILTER records BY (record MATCHES '.*\t2\t.*'OR record MATCHES '.*\t3\t.*'OR record MATCHES '.*\t1\t.*'OR record MATCHES '.*\t4\t.*');

--            PXP2  = FILTER records BY (record MATCHES '.*\tEXEC\t.*'OR record MATCHES '.*\tBIZ\t.*'OR record MATCHES '.*\tSTART_DML\t.*');
--            PXP2 = FOREACH PXP2 GENERATE GET_FIELDS($0) as rec:tuple(mClass:bytearray, rType:bytearray, rName:bytearray, rStatus:bytearray, rDuration:float, rData:bytearray, rDataMap:map[]);
           
--            PXP2 = FOREACH PXP2 GENERATE rec.mClass AS (mClass:bytearray),rec.rType AS (rType:bytearray),rec.rName AS (rName:bytearray),rec.rStatus AS (rStatus:bytearray),rec.rData AS (rData:bytearray),rec.rDataMap AS (rDataMap:map[]);

-- --rec.mClass,rec.rType--,rec.rName,rec.rStatus,rec.rStatus,rec.rDuration,rec.rData,rec.rDataMap,rec.rTime;  --as (rTime:long);
           
           
-- -- rDataMap    
--         GENERATE PXP2;

--          };
   
-- res = DIFF(test1,test2);

-- result = COGROUP test1 by $0, test2 by $0;


T = LIMIT group_data 50;
store T into '$output' using PigStorage(',');  

In [ ]:
-- get_CAL_records, use sample given by Anant, working on 1 records, try to give index to filtered records

%default startDate1 '2020/06/30-00:00:00'
%default endDate1 '2020/06/30-01:00:00'
%default startDate2 '2020/06/30-00:00:00'
%default endDate2 '2020/06/30-01:00:00'

%default corrID 'ac65f81dd963e'

%default server1 'acserv_acriskserv'
%default server2 'activitylogserv'

%default machine 'slcfeel3542b'


table1 = LOAD 'tmp' using RootTransactionLoader('all', 'paypal','$server1', '$machine','$startDate1', '$endDate1', '100','null','null','corr_id_','$corrID') as (env:bytearray, pool:bytearray, machine:bytearray, threadId:int, calClass:bytearray, time:long, type:bytearray,  name:bytearray, status:bytearray, duration:float, dataMap:map[], records:bag{(record:bytearray)}, key:bytearray, colo:bytearray);



test =FOREACH table1 {

           PXP  = FILTER records BY (record MATCHES '.*\tEXEC\t.*'OR record MATCHES '.*\tBIZ\t.*'OR record MATCHES '.*\tSTART_DML\t.*');

           PXP2 = FOREACH PXP GENERATE GET_FIELDS($0) as rec:tuple(mClass:bytearray, rType:bytearray, rName:bytearray, rStatus:bytearray, rDuration:float, rData:bytearray, rDataMap:map[]);
           
           PXP3 = FOREACH PXP2 GENERATE rec.mClass AS (mClass:bytearray),rec.rType AS (rType:bytearray),rec.rName AS (rName:bytearray),rec.rStatus AS (rStatus:bytearray),rec.rData AS (rData:bytearray),rec.rDataMap AS (rDataMap:map[]);
--         working~~~
--         index1 = foreach PXP3 generate CONCAT(rType,rName), mClass,rType,rName,rStatus,rData,rDataMap;


--         GENERATE index1;
      generate PXP;  
         };


                                                                  
T = LIMIT test 100;
store T into '$output' using PigStorage(',');    

In [ ]:
-- for secondary server: get only corre_ids
-- output:all,{(f5cf565d9566d),(40051edd30347),(53ee0d4771acb),(e1e401fcac9fe),(66adec689394c),(5c2cd6baff175),(e92a9c08b8185),(c9e71d008c3be),(af51f1cc91e4d),(7095308b1c679),(f051de3491f34),(6ebe982b4f524),(fe6f8421844a4),(98a7c265640c4),(61ba120702fc4),(8a35f6968d09b),(427a77ecc69ad),(469d807cb9e8c),(67f2d7f1cbdb2),(a507898d1c38c),(b0aaa09d835ff),(3ec85376179a0),(e9102cf9d5461),(939272c248377),(759dea9cf2b31),(a52b395a6a89e),
-- 15 minutes: process in 2 minutes


%default startDate1 '2020/07/20-15:00:00'
%default endDate1 '2020/07/20-15:15:00'
%default server1 'acserv_acriskserv'

table1 = LOAD 'tmp' using RootTransactionLoader('all', 'paypal','$server1', 'null','$startDate1', '$endDate1', '100') as (env:bytearray, pool:bytearray, machine:bytearray, threadId:int, calClass:bytearray, time:long, type:bytearray,  name:bytearray, status:bytearray, duration:float, dataMap:map[], records:bag{(record:bytearray)}, key:bytearray, colo:bytearray);

test1 = FOREACH table1 {
           PXP  = FILTER records BY (record MATCHES '.*\tEXEC\t.*'OR record MATCHES '.*\tBIZ\t.*'OR record MATCHES '.*\tSTART_DML\t.*');
--             PXP = FOREACH PXP GENERATE GET_FIELDS($0) as rec:tuple(mClass:bytearray, rType:bytearray, rName:bytearray, rStatus:bytearray, rDuration:float, rData:bytearray, rDataMap:map[]);
                      
--           PXP = FOREACH PXP GENERATE rec.mClass AS (mClass:bytearray),rec.rType AS (rType:bytearray),rec.rName AS (rName:bytearray),rec.rStatus AS (rStatus:bytearray),rec.rData AS (rData:bytearray),rec.rDataMap AS (rDataMap:map[]);
          GENERATE dataMap#'corr_id_' as (corrID:bytearray) ,PXP;

         };

-- only_not_nulls = FILTER test1 BY corrID is not null;
group_all = GROUP test1 BY corrID;

T = LIMIT group_all 100;
store T into '$output' using PigStorage(',');  

In [ ]:
-- get_CAL_records, use sample given by Anant, trying for 2 records's comparison! Working!

-- {({},no_id)}
-- 1	{({(t,BIZ,KEY_LOOKUP_CALL,,,),(A,EXEC,874948978,0,HOST=nafasstore&timeout=60000&connId=eb065dcb-c3d1-40f5-94a4-b9805ae15e01,[connId#eb065dcb-c3d1-40f5-94a4-b9805ae15e01,HOST#nafasstore,timeout#60000]),(T,BIZ,KEY_LOOKUP_CALL,0,corr_id_=27952c18b&U2VhcmNoRFRPIFtzZWFyY2hUeXBlPUtFWV9MT09LVVAsIGFjb3VudE51bWJlcj1udWxsLCBzdGFydFRpbWU9MCwgZW5kVGltZT0wLCBwYWdlU2l6ZT0wLCBuZXh0UGFnZVRva2VuPTAsIHNlYXJjaFRleHQ9bnVsbCwgaW5kZXhJZD02OCwgYWN0aXZpdHlLZXlEVE9zPVtjb20ucGF5cGFsLmFjdGl2aXR5LmRhdGEubW9kZWwuZHRvLkFjdGl2aXR5S2V5RFRPQDM4NTc4YzA1XSwgc2VhcmNoQWZ0ZXJQYXJhbT1udWxsLCBzZWFyY2hUZXh0RmllbGRzPW51bGwsIHByaW1hcnlPbmx5PWZhbHNlLCBhY2NvdW50U3Vic2NyaXB0aW9uVHlwZXM9bnVsbCwgY29uc3VtZXJGbGFnPWZhbHNlXQ==,[U2VhcmNoRFRPIFtzZWFyY2hUeXBlPUtFWV9MT09LVVAsIGFjb3VudE51bWJlcj1udWxsLCBzdGFydFRpbWU9MCwgZW5kVGltZT0wLCBwYWdlU2l6ZT0wLCBuZXh0UGFnZVRva2VuPTAsIHNlYXJjaFRleHQ9bnVsbCwgaW5kZXhJZD02OCwgYWN0aXZpdHlLZXlEVE9zPVtjb20ucGF5cGFsLmFjdGl2aXR5LmRhdGEubW9kZWwuZHRvLkFjdGl2aXR5S2V5RFRPQDM4NTc4YzA1XSwgc2VhcmNoQWZ0ZXJQYXJhbT1udWxsLCBzZWFyY2hUZXh0RmllbGRzPW51bGwsIHByaW1hcnlPbmx5PWZhbHNlLCBhY2NvdW50U3Vic2NyaXB0aW9uVHlwZXM9bnVsbCwgY29uc3VtZXJGbGFnPWZhbHNlXQ#=,corr_id_#27952c18b])},27952c18b)}
-- 2	{({},2715782873),({(t,BIZ,KEY_LOOKUP_CALL,,,),(A,EXEC,874948978,0,HOST=nafasstore&timeout=60000&connId=db0fe3ca-2716-4eae-a48e-e60f909815e2,[connId#db0fe3ca-2716-4eae-a48e-e60f909815e2,HOST#nafasstore,timeout#60000]),(T,BIZ,KEY_LOOKUP_CALL,0,corr_id_=2715782873&U2VhcmNoRFRPIFtzZWFyY2hUeXBlPUtFWV9MT09LVVAsIGFjb3VudE51bWJlcj1udWxsLCBzdGFydFRpbWU9MCwgZW5kVGltZT0wLCBwYWdlU2l6ZT0wLCBuZXh0UGFnZVRva2VuPTAsIHNlYXJjaFRleHQ9bnVsbCwgaW5kZXhJZD0yMywgYWN0aXZpdHlLZXlEVE9zPVtjb20ucGF5cGFsLmFjdGl2aXR5LmRhdGEubW9kZWwuZHRvLkFjdGl2aXR5S2V5RFRPQDJjZmYyMWVjLCBjb20ucGF5cGFsLmFjdGl2aXR5LmRhdGEubW9kZWwuZHRvLkFjdGl2aXR5S2V5RFRPQDY3MDM0NGQyLCBjb20ucGF5cGFsLmFjdGl2aXR5LmRhdGEubW9kZWwuZHRvLkFjdGl2aXR5S2V5RFRPQDdjMzVkOTMwLCBjb20ucGF5cGFsLmFjdGl2aXR5LmRhdGEubW9kZWwuZHRvLkFjdGl2aXR5S2V5RFRPQDdiZDBmOTdiXSwgc2VhcmNoQWZ0ZXJQYXJhbT1udWxsLCBzZWFyY2hUZXh0RmllbGRzPW51bGwsIHByaW1hcnlPbmx5PWZhbHNlLCBhY2NvdW50U3Vic2NyaXB0aW9uVHlwZXM9bnVsbCwgY29uc3VtZXJGbGFnPWZhbHNlXQ==,[corr_id_#2715782873,U2VhcmNoRFRPIFtzZWFyY2hUeXBlPUtFWV9MT09LVVAsIGFjb3VudE51bWJlcj1udWxsLCBzdGFydFRpbWU9MCwgZW5kVGltZT0wLCBwYWdlU2l6ZT0wLCBuZXh0UGFnZVRva2VuPTAsIHNlYXJjaFRleHQ9bnVsbCwgaW5kZXhJZD0yMywgYWN0aXZpdHlLZXlEVE9zPVtjb20ucGF5cGFsLmFjdGl2aXR5LmRhdGEubW9kZWwuZHRvLkFjdGl2aXR5S2V5RFRPQDJjZmYyMWVjLCBjb20ucGF5cGFsLmFjdGl2aXR5LmRhdGEubW9kZWwuZHRvLkFjdGl2aXR5S2V5RFRPQDY3MDM0NGQyLCBjb20ucGF5cGFsLmFjdGl2aXR5LmRhdGEubW9kZWwuZHRvLkFjdGl2aXR5S2V5RFRPQDdjMzVkOTMwLCBjb20ucGF5cGFsLmFjdGl2aXR5LmRhdGEubW9kZWwuZHRvLkFjdGl2aXR5S2V5RFRPQDdiZDBmOTdiXSwgc2VhcmNoQWZ0ZXJQYXJhbT1udWxsLCBzZWFyY2hUZXh0RmllbGRzPW51bGwsIHByaW1hcnlPbmx5PWZhbHNlLCBhY2NvdW50U3Vic2NyaXB0aW9uVHlwZXM9bnVsbCwgY29uc3VtZXJGbGFnPWZhbHNlXQ#=])},2715782873),({(t,BIZ,KEY_LOOKUP_CALL,,,),(A,EXEC,874948978,0,HOST=nafasstore&timeout=60000&connId=08703bf6-6576-4d3f-91c6-1d06bb5569c2,[connId#08703bf6-6576-4d3f-91c6-1d06bb5569c2,HOST#nafasstore,timeout#60000]),(T,BIZ,KEY_LOOKUP_CALL,0,corr_id_=2715782873&U2VhcmNoRFRPIFtzZWFyY2hUeXBlPUtFWV9MT09LVVAsIGFjb3VudE51bWJlcj1udWxsLCBzdGFydFRpbWU9MCwgZW5kVGltZT0wLCBwYWdlU2l6ZT0wLCBuZXh0UGFnZVRva2VuPTAsIHNlYXJjaFRleHQ9bnVsbCwgaW5kZXhJZD03NywgYWN0aXZpdHlLZXlEVE9zPVtjb20ucGF5cGFsLmFjdGl2aXR5LmRhdGEubW9kZWwuZHRvLkFjdGl2aXR5S2V5RFRPQGEyNDZlMDIsIGNvbS5wYXlwYWwuYWN0aXZpdHkuZGF0YS5tb2RlbC5kdG8uQWN0aXZpdHlLZXlEVE9ANTgxOWE2NSwgY29tLnBheXBhbC5hY3Rpdml0eS5kYXRhLm1vZGVsLmR0by5BY3Rpdml0eUtleURUT0AyM2M3NjE4OCwgY29tLnBheXBhbC5hY3Rpdml0eS5kYXRhLm1vZGVsLmR0by5BY3Rpdml0eUtleURUT0A0Yjc5ODE0NCwgY29tLnBheXBhbC5hY3Rpdml0eS5kYXRhLm1vZGVsLmR0by5BY3Rpdml0eUtleURUT0A2Mjg0YTI1N10sIHNlYXJjaEFmdGVyUGFyYW09bnVsbCwgc2VhcmNoVGV4dEZpZWxkcz1udWxsLCBwcmltYXJ5T25seT1mYWxzZSwgYWNjb3VudFN1YnNjcmlwdGlvblR5cGVzPW51bGwsIGNvbnN1bWVyRmxhZz1mYWxzZV0=,[corr_id_#2715782873,U2VhcmNoRFRPIFtzZWFyY2hUeXBlPUtFWV9MT09LVVAsIGFjb3VudE51bWJlcj1udWxsLCBzdGFydFRpbWU9MCwgZW5kVGltZT0wLCBwYWdlU2l6ZT0wLCBuZXh0UGFnZVRva2VuPTAsIHNlYXJjaFRleHQ9bnVsbCwgaW5kZXhJZD03NywgYWN0aXZpdHlLZXlEVE9zPVtjb20ucGF5cGFsLmFjdGl2aXR5LmRhdGEubW9kZWwuZHRvLkFjdGl2aXR5S2V5RFRPQGEyNDZlMDIsIGNvbS5wYXlwYWwuYWN0aXZpdHkuZGF0YS5tb2RlbC5kdG8uQWN0aXZpdHlLZXlEVE9ANTgxOWE2NSwgY29tLnBheXBhbC5hY3Rpdml0eS5kYXRhLm1vZGVsLmR0by5BY3Rpdml0eUtleURUT0AyM2M3NjE4OCwgY29tLnBheXBhbC5hY3Rpdml0eS5kYXRhLm1vZGVsLmR0by5BY3Rpdml0eUtleURUT0A0Yjc5ODE0NCwgY29tLnBheXBhbC5hY3Rpdml0eS5kYXRhLm1vZGVsLmR0by5BY3Rpdml0eUtleURUT0A2Mjg0YTI1N10sIHNlYXJjaEFmdGVyUGFyYW09bnVsbCwgc2VhcmNoVGV4dEZpZWxkcz1udWxsLCBwcmltYXJ5T25seT1mYWxzZSwgYWNjb3VudFN1YnNjcmlwdGlvblR5cGVzPW51bGwsIGNvbnN1bWVyRmxhZz1mYWxzZV0#])},2715782873)}
-- 3	{({(t,BIZ,KEY_LOOKUP_CALL,,,),(A,EXEC,874948978,0,HOST=nafasstore&timeout=60000&connId=eeefa45c-6b8a-4d37-8d50-b01767de3405,[connId#eeefa45c-6b8a-4d37-8d50-b01767de3405,HOST#nafasstore,timeout#60000]),(T,BIZ,KEY_LOOKUP_CALL,0,corr_id_=3f1c699b98&U2VhcmNoRFRPIFtzZWFyY2hUeXBlPUtFWV9MT09LVVAsIGFjb3VudE51bWJlcj1udWxsLCBzdGFydFRpbWU9MCwgZW5kVGltZT0wLCBwYWdlU2l6ZT0wLCBuZXh0UGFnZVRva2VuPTAsIHNlYXJjaFRleHQ9bnVsbCwgaW5kZXhJZD0xMjEsIGFjdGl2aXR5S2V5RFRPcz1bY29tLnBheXBhbC5hY3Rpdml0eS5kYXRhLm1vZGVsLmR0by5BY3Rpdml0eUtleURUT0AyYWI4NTQ1YV0sIHNlYXJjaEFmdGVyUGFyYW09bnVsbCwgc2VhcmNoVGV4dEZpZWxkcz1udWxsLCBwcmltYXJ5T25seT1mYWxzZSwgYWNjb3VudFN1YnNjcmlwdGlvblR5cGVzPW51bGwsIGNvbnN1bWVyRmxhZz1mYWxzZV0=,[U2VhcmNoRFRPIFtzZWFyY2hUeXBlPUtFWV9MT09LVVAsIGFjb3VudE51bWJlcj1udWxsLCBzdGFydFRpbWU9MCwgZW5kVGltZT0wLCBwYWdlU2l6ZT0wLCBuZXh0UGFnZVRva2VuPTAsIHNlYXJjaFRleHQ9bnVsbCwgaW5kZXhJZD0xMjEsIGFjdGl2aXR5S2V5RFRPcz1bY29tLnBheXBhbC5hY3Rpdml0eS5kYXRhLm1vZGVsLmR0by5BY3Rpdml0eUtleURUT0AyYWI4NTQ1YV0sIHNlYXJjaEFmdGVyUGFyYW09bnVsbCwgc2VhcmNoVGV4dEZpZWxkcz1udWxsLCBwcmltYXJ5T25seT1mYWxzZSwgYWNjb3VudFN1YnNjcmlwdGlvblR5cGVzPW51bGwsIGNvbnN1bWVyRmxhZz1mYWxzZV0#,corr_id_#3f1c699b98])},3f1c699b98)}
-- 4	{({(t,BIZ,KEY_LOOKUP_CALL,,,),(A,EXEC,874948978,0,HOST=nafasstore&timeout=60000&connId=c32cba98-41b8-4af0-a2bd-188212a102c4,[connId#c32cba98-41b8-4af0-a2bd-188212a102c4,HOST#nafasstore,timeout#60000]),(T,BIZ,KEY_LOOKUP_CALL,0,corr_id_=4cc1336b4e&U2VhcmNoRFRPIFtzZWFyY2hUeXBlPUtFWV9MT09LVVAsIGFjb3VudE51bWJlcj1udWxsLCBzdGFydFRpbWU9MCwgZW5kVGltZT0wLCBwYWdlU2l6ZT0wLCBuZXh0UGFnZVRva2VuPTAsIHNlYXJjaFRleHQ9bnVsbCwgaW5kZXhJZD04NCwgYWN0aXZpdHlLZXlEVE9zPVtjb20ucGF5cGFsLmFjdGl2aXR5LmRhdGEubW9kZWwuZHRvLkFjdGl2aXR5S2V5RFRPQDYxMGJjOGQzXSwgc2VhcmNoQWZ0ZXJQYXJhbT1udWxsLCBzZWFyY2hUZXh0RmllbGRzPW51bGwsIHByaW1hcnlPbmx5PWZhbHNlLCBhY2NvdW50U3Vic2NyaXB0aW9uVHlwZXM9bnVsbCwgY29uc3VtZXJGbGFnPWZhbHNlXQ==,[U2VhcmNoRFRPIFtzZWFyY2hUeXBlPUtFWV9MT09LVVAsIGFjb3VudE51bWJlcj1udWxsLCBzdGFydFRpbWU9MCwgZW5kVGltZT0wLCBwYWdlU2l6ZT0wLCBuZXh0UGFnZVRva2VuPTAsIHNlYXJjaFRleHQ9bnVsbCwgaW5kZXhJZD04NCwgYWN0aXZpdHlLZXlEVE9zPVtjb20ucGF5cGFsLmFjdGl2aXR5LmRhdGEubW9kZWwuZHRvLkFjdGl2aXR5S2V5RFRPQDYxMGJjOGQzXSwgc2VhcmNoQWZ0ZXJQYXJhbT1udWxsLCBzZWFyY2hUZXh0RmllbGRzPW51bGwsIHByaW1hcnlPbmx5PWZhbHNlLCBhY2NvdW50U3Vic2NyaXB0aW9uVHlwZXM9bnVsbCwgY29uc3VtZXJGbGFnPWZhbHNlXQ#=,corr_id_#4cc1336b4e])},4cc1336b4e),({},4cc1336b4e)}
-- 5	{({(t,BIZ,KEY_LOOKUP_CALL,,,),(A,EXEC,874948978,0,HOST=nafasstore&timeout=60000&connId=9a5da984-2398-4d29-be82-973dade358ee,[connId#9a5da984-2398-4d29-be82-973dade358ee,HOST#nafasstore,timeout#60000]),(T,BIZ,KEY_LOOKUP_CALL,0,corr_id_=622151f4dc&U2VhcmNoRFRPIFtzZWFyY2hUeXBlPUtFWV9MT09LVVAsIGFjb3VudE51bWJlcj1udWxsLCBzdGFydFRpbWU9MCwgZW5kVGltZT0wLCBwYWdlU2l6ZT0wLCBuZXh0UGFnZVRva2VuPTAsIHNlYXJjaFRleHQ9bnVsbCwgaW5kZXhJZD00OCwgYWN0aXZpdHlLZXlEVE9zPVtjb20ucGF5cGFsLmFjdGl2aXR5LmRhdGEubW9kZWwuZHRvLkFjdGl2aXR5S2V5RFRPQDZlMmViODc0LCBjb20ucGF5cGFsLmFjdGl2aXR5LmRhdGEubW9kZWwuZHRvLkFjdGl2aXR5S2V5RFRPQDNkNWIxMTgzLCBjb20ucGF5cGFsLmFjdGl2aXR5LmRhdGEubW9kZWwuZHRvLkFjdGl2aXR5S2V5RFRPQDRkOTlhMmY3XSwgc2VhcmNoQWZ0ZXJQYXJhbT1udWxsLCBzZWFyY2hUZXh0RmllbGRzPW51bGwsIHByaW1hcnlPbmx5PWZhbHNlLCBhY2NvdW50U3Vic2NyaXB0aW9uVHlwZXM9bnVsbCwgY29uc3VtZXJGbGFnPWZhbHNlXQ==,[corr_id_#622151f4dc,U2VhcmNoRFRPIFtzZWFyY2hUeXBlPUtFWV9MT09LVVAsIGFjb3VudE51bWJlcj1udWxsLCBzdGFydFRpbWU9MCwgZW5kVGltZT0wLCBwYWdlU2l6ZT0wLCBuZXh0UGFnZVRva2VuPTAsIHNlYXJjaFRleHQ9bnVsbCwgaW5kZXhJZD00OCwgYWN0aXZpdHlLZXlEVE9zPVtjb20ucGF5cGFsLmFjdGl2aXR5LmRhdGEubW9kZWwuZHRvLkFjdGl2aXR5S2V5RFRPQDZlMmViODc0LCBjb20ucGF5cGFsLmFjdGl2aXR5LmRhdGEubW9kZWwuZHRvLkFjdGl2aXR5S2V5RFRPQDNkNWIxMTgzLCBjb20ucGF5cGFsLmFjdGl2aXR5LmRhdGEubW9kZWwuZHRvLkFjdGl2aXR5S2V5RFRPQDRkOTlhMmY3XSwgc2VhcmNoQWZ0ZXJQYXJhbT1udWxsLCBzZWFyY2hUZXh0RmllbGRzPW51bGwsIHByaW1hcnlPbmx5PWZhbHNlLCBhY2NvdW50U3Vic2NyaXB0aW9uVHlwZXM9bnVsbCwgY29uc3VtZXJGbGFnPWZhbHNlXQ#=])},622151f4dc),({(t,BIZ,KEY_LOOKUP_CALL,,,),(A,EXEC,874948978,0,HOST=nafasstore&timeout=60000&connId=fca004fa-e63e-4ac1-9097-c973e34ea80b,[connId#fca004fa-e63e-4ac1-9097-c973e34ea80b,HOST#nafasstore,timeout#60000]),(T,BIZ,KEY_LOOKUP_CALL,0,corr_id_=622151f4dc&U2VhcmNoRFRPIFtzZWFyY2hUeXBlPUtFWV9MT09LVVAsIGFjb3VudE51bWJlcj1udWxsLCBzdGFydFRpbWU9MCwgZW5kVGltZT0wLCBwYWdlU2l6ZT0wLCBuZXh0UGFnZVRva2VuPTAsIHNlYXJjaFRleHQ9bnVsbCwgaW5kZXhJZD02MywgYWN0aXZpdHlLZXlEVE9zPVtjb20ucGF5cGFsLmFjdGl2aXR5LmRhdGEubW9kZWwuZHRvLkFjdGl2aXR5S2V5RFRPQDFkNGEzZTY1LCBjb20ucGF5cGFsLmFjdGl2aXR5LmRhdGEubW9kZWwuZHRvLkFjdGl2aXR5S2V5RFRPQDQwMmYyMWUyLCBjb20ucGF5cGFsLmFjdGl2aXR5LmRhdGEubW9kZWwuZHRvLkFjdGl2aXR5S2V5RFRPQDI5ZWYzZWQ3LCBjb20ucGF5cGFsLmFjdGl2aXR5LmRhdGEubW9kZWwuZHRvLkFjdGl2aXR5S2V5RFRPQDVmOTUzZGU1LCBjb20ucGF5cGFsLmFjdGl2aXR5LmRhdGEubW9kZWwuZHRvLkFjdGl2aXR5S2V5RFRPQDEzZTA4OTY0XSwgc2VhcmNoQWZ0ZXJQYXJhbT1udWxsLCBzZWFyY2hUZXh0RmllbGRzPW51bGwsIHByaW1hcnlPbmx5PWZhbHNlLCBhY2NvdW50U3Vic2NyaXB0aW9uVHlwZXM9bnVsbCwgY29uc3VtZXJGbGFnPWZhbHNlXQ==,[corr_id_#622151f4dc,U2VhcmNoRFRPIFtzZWFyY2hUeXBlPUtFWV9MT09LVVAsIGFjb3VudE51bWJlcj1udWxsLCBzdGFydFRpbWU9MCwgZW5kVGltZT0wLCBwYWdlU2l6ZT0wLCBuZXh0UGFnZVRva2VuPTAsIHNlYXJjaFRleHQ9bnVsbCwgaW5kZXhJZD02MywgYWN0aXZpdHlLZXlEVE9zPVtjb20ucGF5cGFsLmFjdGl2aXR5LmRhdGEubW9kZWwuZHRvLkFjdGl2aXR5S2V5RFRPQDFkNGEzZTY1LCBjb20ucGF5cGFsLmFjdGl2aXR5LmRhdGEubW9kZWwuZHRvLkFjdGl2aXR5S2V5RFRPQDQwMmYyMWUyLCBjb20ucGF5cGFsLmFjdGl2aXR5LmRhdGEubW9kZWwuZHRvLkFjdGl2aXR5S2V5RFRPQDI5ZWYzZWQ3LCBjb20ucGF5cGFsLmFjdGl2aXR5LmRhdGEubW9kZWwuZHRvLkFjdGl2aXR5S2V5RFRPQDVmOTUzZGU1LCBjb20ucGF5cGFsLmFjdGl2aXR5LmRhdGEubW9kZWwuZHRvLkFjdGl2aXR5S2V5RFRPQDEzZTA4OTY0XSwgc2VhcmNoQWZ0ZXJQYXJhbT1udWxsLCBzZWFyY2hUZXh0RmllbGRzPW51bGwsIHByaW1hcnlPbmx5PWZhbHNlLCBhY2NvdW50U3Vic2NyaXB0aW9uVHlwZXM9bnVsbCwgY29uc3VtZXJGbGFnPWZhbHNlXQ#=])},622151f4dc),({},622151f4dc)}



%default startDate1 '2020/07/08-15:00:00'
%default endDate1 '2020/07/08-16:00:00'
%default startDate2 '2020/07/08-15:00:00'
%default endDate2 '2020/07/08-16:00:00'

%default corrID1 'slcfeel3542b'
-- %default corrID2 'bee179d7bbdf5'

%default server1 'acserv_acriskserv'
%default server2 'activitydataserv'

-- %default machine 'slcfeel6412b'

-- GET_SQL	Gets SQL statement given a SQL hash	GET_SQL(hash) as sql:bytearray
-- GET_SQL_NAME	Gets SQL's name given a SQL hash	GET_SQL_NAME(hash) as name:chararray


-- table1 = LOAD 'tmp' using RootTransactionLoader('all', 'paypal','$server1', '$machine','$startDate1', '$endDate1', '100','null','null','corr_id_','$corrID1') as (env:bytearray, pool:bytearray, machine:bytearray, threadId:int, calClass:bytearray, time:long, type:bytearray,  name:bytearray, status:bytearray, duration:float, dataMap:map[], records:bag{(record:bytearray)}, key:bytearray, colo:bytearray);

table1 = LOAD 'tmp' using RootTransactionLoader('all', 'paypal','$server1', 'null','$startDate1', '$endDate1', '100') as (env:bytearray, pool:bytearray, machine:bytearray, threadId:int, calClass:bytearray, time:long, type:bytearray,  name:bytearray, status:bytearray, duration:float, dataMap:map[], records:bag{(record:bytearray)}, key:bytearray, colo:bytearray);

table2 = LOAD 'tmp' using RootTransactionLoader('all', 'paypal','$server2', 'null','$startDate2', '$endDate2', '100') as (env:bytearray, pool:bytearray, machine:bytearray, threadId:int, calClass:bytearray, time:long, type:bytearray,  name:bytearray, status:bytearray, duration:float, dataMap:map[], records:bag{(record:bytearray)}, key:bytearray, colo:bytearray);

-- records_all = Group table1 All;
-- rec_count = FOREACH records_all Generate COUNT(table1);

-- comb = UNION table1,table2;
-- test2 = foreach table1 generate CONCAT(name, '_',status);

test1 = FOREACH table1 {
           PXP  = FILTER records BY (record MATCHES '.*\tEXEC\t.*'OR record MATCHES '.*\tBIZ\t.*'OR record MATCHES '.*\tSTART_DML\t.*');
            PXP = FOREACH PXP GENERATE GET_FIELDS($0) as rec:tuple(mClass:bytearray, rType:bytearray, rName:bytearray, rStatus:bytearray, rDuration:float, rData:bytearray, rDataMap:map[]);
                      
          PXP = FOREACH PXP GENERATE rec.mClass AS (mClass:bytearray),rec.rType AS (rType:bytearray),rec.rName AS (rName:bytearray),rec.rStatus AS (rStatus:bytearray),rec.rData AS (rData:bytearray),rec.rDataMap AS (rDataMap:map[]);
          GENERATE PXP,dataMap#'corr_id_' as (corrID:bytearray);

         };
   
-- group_data1 = GROUP test1 by corrID;

test2 =FOREACH table2 {
--            PXP  = FILTER records BY (record MATCHES '.*\t2\t.*'OR record MATCHES '.*\t3\t.*'OR record MATCHES '.*\t1\t.*'OR record MATCHES '.*\t4\t.*');

           PXP2  = FILTER records BY (record MATCHES '.*\tEXEC\t.*'OR record MATCHES '.*\tBIZ\t.*'OR record MATCHES '.*\tSTART_DML\t.*');
           PXP2 = FOREACH PXP2 GENERATE GET_FIELDS($0) as rec:tuple(mClass:bytearray, rType:bytearray, rName:bytearray, rStatus:bytearray, rDuration:float, rData:bytearray, rDataMap:map[]);
           
           PXP2 = FOREACH PXP2 GENERATE rec.mClass AS (mClass:bytearray),rec.rType AS (rType:bytearray),rec.rName AS (rName:bytearray),rec.rStatus AS (rStatus:bytearray),rec.rData AS (rData:bytearray),rec.rDataMap AS (rDataMap:map[]);

          GENERATE dataMap#'corr_id_' as (corrID:bytearray);


         };
-- group_data2 = GROUP test2 by corrID;

cogroup_data = COGROUP test1 by corrID, test2 by corrID;

diff_data = FOREACH cogroup_data GENERATE DIFF(test1,test2);
-- result = COGROUP test1 by $0, test2 by $0;


T = LIMIT diff_data 600;
store T into '$output' using PigStorage(',');  

In [ ]:
-- test diff on 1 correlation id, but 2 service

%default startDate1 '2020/07/20-21:00:00'
%default endDate1 '2020/07/20-22:00:00'
%default startDate2 '2020/07/20-21:00:00'
%default endDate2 '2020/07/20-22:00:00'

%default corrID1 '7baf4b0653052'
-- %default corrID2 'bee179d7bbdf5'

%default server1 'acserv_acriskserv'
%default server2 ' compliancepaymentserv'

-- %default machine 'slcfeel6412b'

-- GET_SQL	Gets SQL statement given a SQL hash	GET_SQL(hash) as sql:bytearray
-- GET_SQL_NAME	Gets SQL's name given a SQL hash	GET_SQL_NAME(hash) as name:chararray

-- table2 = LOAD 'tmp' using RootTransactionLoader('all', 'paypal','$server1', '$machine','$startDate1', '$endDate1', '100','null','null','corr_id_','$corrID1') as (env:bytearray, pool:bytearray, machine:bytearray, threadId:int, calClass:bytearray, time:long, type:bytearray,  name:bytearray, status:bytearray, duration:float, dataMap:map[], records:bag{(record:bytearray)}, key:bytearray, colo:bytearray);

table1 = LOAD 'tmp' using RootTransactionLoader('all', 'paypal','$server1', 'null','$startDate1', '$endDate1', '100','null','null','corr_id_','$corrID1') as (env:bytearray, pool:bytearray, machine:bytearray, threadId:int, calClass:bytearray, time:long, type:bytearray,  name:bytearray, status:bytearray, duration:float, dataMap:map[], records:bag{(record:bytearray)}, key:bytearray, colo:bytearray);

table2 = LOAD 'tmp' using RootTransactionLoader('all', 'paypal','$server1', 'null','$startDate1', '$endDate1', '100','null','null','corr_id_','$corrID1') as (env:bytearray, pool:bytearray, machine:bytearray, threadId:int, calClass:bytearray, time:long, type:bytearray,  name:bytearray, status:bytearray, duration:float, dataMap:map[], records:bag{(record:bytearray)}, key:bytearray, colo:bytearray);

-- table1 = LOAD 'tmp' using RootTransactionLoader('all', 'paypal','$server1', 'null','$startDate1', '$endDate1', '100') as (env:bytearray, pool:bytearray, machine:bytearray, threadId:int, calClass:bytearray, time:long, type:bytearray,  name:bytearray, status:bytearray, duration:float, dataMap:map[], records:bag{(record:bytearray)}, key:bytearray, colo:bytearray);

-- table2 = LOAD 'tmp' using RootTransactionLoader('all', 'paypal','$server2', 'null','$startDate2', '$endDate2', '100') as (env:bytearray, pool:bytearray, machine:bytearray, threadId:int, calClass:bytearray, time:long, type:bytearray,  name:bytearray, status:bytearray, duration:float, dataMap:map[], records:bag{(record:bytearray)}, key:bytearray, colo:bytearray);

-- records_all = Group table1 All;
-- rec_count = FOREACH records_all Generate COUNT(table1);

-- comb = UNION table1,table2;
-- test2 = foreach table1 generate CONCAT(name, '_',status);

test1 = FOREACH table1 {
           PXP  = FILTER records BY (record MATCHES '.*\tEXEC\t.*'OR record MATCHES '.*\tBIZ\t.*'OR record MATCHES '.*\tSTART_DML\t.*');
            PXP = FOREACH PXP GENERATE GET_FIELDS($0) as rec:tuple(mClass:bytearray, rType:bytearray, rName:bytearray, rStatus:bytearray, rDuration:float, rData:bytearray, rDataMap:map[]);
                      
          PXP = FOREACH PXP GENERATE rec.mClass AS (mClass:bytearray),rec.rType AS (rType:bytearray),rec.rName AS (rName:bytearray),rec.rStatus AS (rStatus:bytearray),rec.rData AS (rData:bytearray),rec.rDataMap AS (rDataMap:map[]);
--           index1 = foreach PXP generate CONCAT(rType,rName) as key, mClass,rType,rName,rStatus,rData,rDataMap;

          GENERATE PXP,rName as key;
--           GENERATE index1;

         };
   
-- group_data1 = GROUP test1 by corrID;

test2 =FOREACH table2 {
--            PXP  = FILTER records BY (record MATCHES '.*\t2\t.*'OR record MATCHES '.*\t3\t.*'OR record MATCHES '.*\t1\t.*'OR record MATCHES '.*\t4\t.*');

           PXP2  = FILTER records BY (record MATCHES '.*\tEXEC\t.*'OR record MATCHES '.*\tBIZ\t.*'OR record MATCHES '.*\tSTART_DML\t.*');
           PXP2 = FOREACH PXP2 GENERATE GET_FIELDS($0) as rec:tuple(mClass:bytearray, rType:bytearray, rName:bytearray, rStatus:bytearray, rDuration:float, rData:bytearray, rDataMap:map[]);
           
           PXP2 = FOREACH PXP2 GENERATE rec.mClass AS (mClass:bytearray),rec.rType AS (rType:bytearray),rec.rName AS (rName:bytearray),rec.rStatus AS (rStatus:bytearray),rec.rData AS (rData:bytearray),rec.rDataMap AS (rDataMap:map[]);
--           index2 = foreach PXP2 generate CONCAT(rType,rName) as key, mClass,rType,rName,rStatus,rData,rDataMap;
--           GENERATE PXP2,dataMap#'corr_id_' as (corrID:bytearray);
          GENERATE PXP2,rName as key;

--           GENERATE index2;


         };
-- group_data2 = GROUP test2 by corrID;

cogroup_data = COGROUP test1 by key, test2 by key;

diff_data = FOREACH cogroup_data GENERATE DIFF(test1,test2);
-- result = COGROUP test1 by $0, test2 by $0;


T = LIMIT cogroup_data 600;
store T into '$output' using PigStorage(',');  